In [1]:
import pandas as pd
import plotly.graph_objects as go

class LRUPageReplacement:
    def __init__(self, num_frames):
        self.num_frames = num_frames
        self.frames = []
        self.page_faults = 0
        self.page_hits = 0
        self.page_sequence = []
        self.page_steps = []

    def access_page(self, page):
        step = {"Page": page, "Hit/Fault": "Fault"}
        
        # Track the current state of frames
        frame_state = list(self.frames)
        
        if page in self.frames:
            # Page hit: move the page to the most recently used
            self.page_hits += 1
            step["Hit/Fault"] = "Hit"
            self.page_steps.append((page, frame_state, "Hit"))
            # Move the page to the end (most recently used)
            self.frames.remove(page)
            self.frames.append(page)
            return
        
        # Page fault: add the page or replace the least recently used page
        if len(self.frames) < self.num_frames:
            self.frames.append(page)
        else:
            self.frames.pop(0)  # Remove the least recently used page
            self.frames.append(page)
        
        self.page_faults += 1
        self.page_steps.append((page, frame_state, "Fault"))

    def simulate(self, page_sequence):
        self.page_sequence = page_sequence
        for page in page_sequence:
            self.access_page(page)

    def print_results(self):
        print(f"Page sequence: {self.page_sequence}")
        print(f"Page hits: {self.page_hits}")
        print(f"Page faults: {self.page_faults}")
        print(f"Hit ratio: {self.page_hits / len(self.page_sequence):.2f}")
        print(f"Fault ratio: {self.page_faults / len(self.page_sequence):.2f}")

    def display_table(self):
        # Create DataFrame with steps for displaying
        data = []
        for page, frame_state, hit_fault in self.page_steps:
            row = {'Page': page}
            # Fill in frame columns (F1, F2, ..., Fn)
            for i in range(self.num_frames):
                row[f"F{i+1}"] = frame_state[i] if i < len(frame_state) else ""
            row['Hit/Fault'] = hit_fault
            data.append(row)

        df = pd.DataFrame(data)

        # Create Plotly table
        fig = go.Figure(data=[go.Table(
            header=dict(values=["Page"] + [f"F{i+1}" for i in range(self.num_frames)] + ["Hit/Fault"],
                        fill_color='paleturquoise', align='center'),
            cells=dict(values=[df['Page']] + [df[f"F{i+1}"] for i in range(self.num_frames)] + [df['Hit/Fault']],
                       fill_color='lavender', align='center'))
        ])

        # Update layout for better presentation
        fig.update_layout(
            title="LRU Page Replacement Process",
            height=400,
            width=800
        )

        # Show the table
        fig.show()


# Get user input for the simulation
num_frames = int(input("Enter the number of frames: "))
page_sequence_input = input("Enter the page reference string (space-separated): ")
page_sequence = list(map(int, page_sequence_input.split()))

# Initialize the LRU Page Replacement
lru = LRUPageReplacement(num_frames)

# Simulate the page replacement
lru.simulate(page_sequence)

# Print results
lru.print_results()

# Display the table showing the page replacement process
lru.display_table()

Page sequence: [1, 2, 3, 1, 2, 4, 5]
Page hits: 2
Page faults: 5
Hit ratio: 0.29
Fault ratio: 0.71
